# Introduction: A Working MNIST Classifier

- Understanding softmax and NLL: https://ljvmiranda921.github.io/notebook/2017/08/13/softmax-and-the-negative-log-likelihood/
- Visualizing MNIST (Chris Olah): http://colah.github.io/posts/2014-10-Visualizing-MNIST/
- Pytorch: https://pytorch.org/
- Pytorch samples: https://github.com/pytorch/examples

In [ ]:
import torch as t
import torch.nn as nn
import torch.nn.functional as f
import torchvision as tv

![](img/mnist.png)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        # [64, 1, 28, 28]
        x = f.relu(self.conv1(x)) # [64, 20, 24, 24]
        x = f.max_pool2d(x, 2, 2) # [64, 20, 12, 12]
        x = f.relu(self.conv2(x)) # [64, 50, 8, 8]
        x = f.max_pool2d(x, 2, 2) # [64, 50, 4, 4]
        x = x.view(-1, 4*4*50) # [64, 800]
        x = f.relu(self.fc1(x)) # [64, 500]
        x = self.fc2(x) # [64, 10]
        x = f.log_softmax(x, dim=1) # [64, 10]
        return x

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = f.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        # item(): https://pytorch.org/docs/stable/tensors.html#torch.Tensor.item
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with t.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += f.nll_loss(output, target, reduction='sum').item() # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # Get the index of the max log-probability
            # eq(): https://pytorch.org/docs/stable/torch.html#torch.eq
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
device = t.device("cuda" if t.cuda.is_available() else "cpu")

transform = tv.transforms.Compose([tv.transforms.ToTensor(), tv.transforms.Normalize((0.1307,), (0.3081,))])

train_set = tv.datasets.MNIST('.', train=True, transform = transform)
test_set = tv.datasets.MNIST('.', train=False, transform = transform)
train_loader = t.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = t.utils.data.DataLoader(test_set, batch_size=1000, shuffle=True)

model = Net().to(device)
optimizer = t.optim.SGD(model.parameters(), lr=0.01)

for epoch in range(1, 10 + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)